# Learn OpenAI Whisper - Chapter 4
## Fine-tune Whisper with Hugging Face transformers

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1LADNomT0JUBCsopU6r_NsZfOaNiz2N3h)

In this Colab, we present a step-by-step guide on how to fine-tune Whisper
for any multilingual ASR dataset using Hugging Face 🤗 Transformers.

## Prepare environment

First of all, let's try to secure a decent GPU for our Colab! Unfortunately, it's becoming much harder to get access to a good GPU with the free version of Google Colab. However, with Google Colab Pro one should have no issues in being allocated a V100 or P100 GPU.

To get a GPU, click _Runtime_ -> _Change runtime type_, then change _Hardware accelerator_ from _None_ to _GPU_.

We can verify that we've been assigned a GPU and view its specifications:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

We'll employ several popular Python packages to fine-tune the Whisper model.
We'll use `datasets` to download and prepare our training data and
`transformers` to load and train our Whisper model. We'll also require
the `soundfile` package to pre-process audio files, `evaluate` and `jiwer` to
assess the performance of our model. Finally, we'll
use `gradio` to build a flashy demo of our fine-tuned model.

In [ ]:
!pip install -q datasets>=2.6.1
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q librosa
!pip install -q evaluate>=0.30
!pip install -q jiwer
!pip install -q gradio

In [ ]:
!pip install -q accelerate -U

We highly recommend directly uploading model checkpoints to the [Hugging Face Hub](https://huggingface.co/) while you train. The Hub offers several benefits:
- **Version Control**: Ensures every model checkpoint is preserved throughout the training process.
- **Tensorboard Logs**: Allows for the monitoring of key metrics as training progresses.
- **Model Cards**: Provides a space to explain the model's functionality and its appropriate applications.
- **Community Engagement**: Facilitates sharing and collaboration with a wider community.

Connecting this notebook with the Hub is quite simple – all it requires is your Hub authentication token when asked. You can locate your Hub authentication token [here](https://huggingface.co/settings/tokens).

**IMPORTANT**
Make sure that your Hugging Face token was created with role `write` because the notebook will be saving the model to the Hugging Face repository.

If you need instructions on how to create a Hugging Face tocken, please check out the [notebook for Chapter 3](https://github.com/PacktPublishing/Learn-OpenAI-Whisper/blob/main/Chapter03/LOAIW_ch03_working_with_audio_data_via_Hugging_Face.ipynb).


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

**IMPORTANT:**
The output of `whoami90` shoud show `'role': 'write'` in your `auth` keys.. If it does not, then you might need to create a new token with role `write`, rerun the previous cell to execute `notebook_login()`, and load the new token.  

<img src="https://github.com/PacktPublishing/Learn-OpenAI-Whisper/raw/main/Chapter04/hugging_face_whoami.png">


In [ ]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

## Load dataset

Utilizing 🤗 Datasets makes the task of downloading and preparing data remarkably straightforward.
You can obtain and ready the Common Voice splits with a mere single line of code.

Initially, confirm that you've agreed to the usage terms on the Hugging Face Hub: [mozilla-foundation/common_voice_11_0](https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0). After agreeing to the terms, you gain complete access to the dataset, enabling you to download the data to your local environment.

Given the scarcity of resources for Hindi, we plan to merge the `train` and `validation` splits, resulting in around 8 hours of training material. The 4 hours of `test` data will serve as our separate test set:

In [ ]:
from datasets import load_dataset, DatasetDict

common_voice = DatasetDict()

common_voice["train"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="train+validation", use_auth_token=True)
common_voice["test"] = load_dataset("mozilla-foundation/common_voice_11_0", "hi", split="test", use_auth_token=True)

print(common_voice)

The majority of ASR datasets offer just the input audio clips (`audio`) and their transcribed text (`sentence`). However, Common Voice includes extra metadata, like `accent` and `locale`, which aren't necessary for ASR tasks. To maintain the notebook's broad applicability, we focus solely on the input audio and the transcribed text for fine-tuning, omitting the extra metadata:

In [ ]:
common_voice = common_voice.remove_columns(["accent", "age", "client_id", "down_votes", "gender", "locale", "path", "segment", "up_votes"])

print(common_voice)

## Prepare feature extractor, tokenizer and data

The ASR process can be broken down into three main components:
1. A feature extractor that prepares the raw audio inputs
2. The model that executes the sequence-to-sequence transformation
3. A tokenizer that converts the model's outputs into text format

Within 🤗 Transformers, the Whisper model comes with its own feature extractor and tokenizer, named [WhisperFeatureExtractor](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperFeatureExtractor) and [WhisperTokenizer](https://huggingface.co/docs/transformers/main/model_doc/whisper#transformers.WhisperTokenizer), respectively.

Let's dive into the specifics of configuring both the feature extractor and tokenizer step by step!

### Load WhisperFeatureExtractor

The Whisper feature extractor conducts two key tasks:
1. Adjusts the length of the audio inputs to 30 seconds: it extends any audio shorter than 30 seconds with silence (zeros) to reach 30 seconds, and it cuts down any audio longer than 30 seconds to the same duration.
2. Transforms the audio inputs into _log-Mel spectrogram_ input features, which visually represent the audio and match the input format the Whisper model anticipates.

<figure>
<img src="https://raw.githubusercontent.com/sanchit-gandhi/notebooks/main/spectrogram.jpg" alt="Trulli" style="width:100%">
<figcaption align = "center"><b>Figure 2:</b> Conversion of sampled audio array to log-Mel spectrogram.
Left: sampled 1-dimensional audio signal. Right: corresponding log-Mel spectrogram. Figure source:
<a href="https://ai.googleblog.com/2019/04/specaugment-new-data-augmentation.html">Google SpecAugment Blog</a>.
</figcaption>

We'll load the feature extractor from the pre-trained checkpoint with the default values:

In [ ]:
from transformers import WhisperFeatureExtractor

feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

### Load WhisperTokenizer

The Whisper model generates a series of _token ids_. Each token id is then translated into its respective text string by the tokenizer. For the Hindi language, we are able to utilize the pre-trained tokenizer directly for fine-tuning without needing any adjustments. All that's required is to define the target language and the task at hand. This instruction guides the tokenizer to prepend the language and task tokens at the beginning of the encoded label sequences:

In [ ]:
from transformers import WhisperTokenizer

tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

### Combine to create a WhisperProcessor

To make the use of the feature extractor and tokenizer more straightforward, we can bundle them together into a unified `WhisperProcessor` class. This processor amalgamates functionalities from both the `WhisperFeatureExtractor` and `WhisperTokenizer`, allowing it to be applied to both the audio inputs and model outputs as needed. By adopting this approach, we simplify the components we need to manage during training to just two items: the `processor` and the `model`:

In [ ]:
from transformers import WhisperProcessor

processor = WhisperProcessor.from_pretrained("openai/whisper-small", language="Hindi", task="transcribe")

### Prepare data

Let's print the first example of the Common Voice dataset to see
what form the data is in:

In [ ]:
print(common_voice["train"][0])

Given that our input audio has a sampling rate of 48kHz, it's essential to _downsample_ it to 16kHz before feeding it into the Whisper feature extractor, since 16kHz is the sampling rate the Whisper model expects.

We will adjust the audio inputs to the appropriate sampling rate using the dataset's [`cast_column`](https://huggingface.co/docs/datasets/package_reference/main_classes.html?highlight=cast_column#datasets.DatasetDict.cast_column) function. This process doesn't alter the audio files directly. Instead, it instructs the `datasets` library to resample the audio samples _on the fly_ upon their initial access:

In [ ]:
from datasets import Audio

common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))

Re-loading the first audio sample in the Common Voice dataset will resample
it to the desired sampling rate:

In [ ]:
print(common_voice["train"][0])

Now, let's craft a function to get our data set for the model:
1. By invoking `batch["audio"]`, we load and resample the audio data. As mentioned before, 🤗 Datasets handles the resampling as needed automatically.
2. The feature extractor is utilized to derive the log-Mel spectrogram input features from our one-dimensional audio array.
3. The transcriptions are converted into label ids by employing the tokenizer.

In [ ]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

We're able to implement the data preparation function across all training examples by utilizing the dataset's `.map` method. The `num_proc` parameter determines the number of CPU cores employed. Configuring `num_proc` to be greater than 1 activates multiprocessing. Should the `.map` method experience delays while using multiprocessing, adjusting `num_proc` to 1 will ensure the dataset is processed in a sequential manner.

In [ ]:
common_voice = common_voice.map(prepare_dataset, remove_columns=common_voice.column_names["train"], num_proc=2)

## Training and evaluation

With our data now prepared, we're set to embark on setting up the training pipeline. The [🤗 Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) simplifies much of this process for us. Our tasks include:

- Establishing a data collator: This function gathers our processed data and organizes it into PyTorch tensors, making them ready for the model.

- Choosing evaluation metrics: For evaluation purposes, the model's performance will be assessed using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric. We'll need a `compute_metrics` function that will manage this evaluation.

- Loading a pre-trained checkpoint: A pre-trained checkpoint is loaded and set up for the training phase.

- Configuring the training setup: The 🤗 Trainer relies on this configuration to outline the training protocol.

After fine-tuning the model, we'll assess its performance on the test dataset to ensure it's been properly trained for speech transcription in Hindi.

### Define a data collator

The data collator for a sequence-to-sequence speech model distinctively processes `input_features` and `labels` separately: `input_features` are managed with the feature extractor while `labels` are dealt with by the tokenizer.

`Input_features` get padded to 30 seconds and transformed into a log-Mel spectrogram of a uniform size through the feature extractor's intervention. The next step involves converting these `input_features` into batched PyTorch tensors, accomplished using the feature extractor's `.pad` method and setting `return_tensors=pt`.

On the flip side, `labels` remain un-padded. The initial action is to pad these sequences to the batch's maximal length via the tokenizer's `.pad` method. Subsequently, padding tokens are substituted with `-100` to ensure they are excluded from loss calculations. Additionally, we remove the BOS token from the start of the label sequence since it will be added later in the training phase.

The `WhisperProcessor` we previously established can be utilized to conduct both feature extraction and tokenization tasks:

In [ ]:
import torch

from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

Let's initialise the data collator we've just defined:

In [ ]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [ ]:
print(common_voice)

### Evaluation metrics

We'll employ the word error rate (WER) metric, widely considered the standard for evaluating ASR systems. For additional details, you can consult the WER [documentation](https://huggingface.co/metrics/wer). The WER metric will be sourced from 🤗 Evaluate:

In [ ]:
import evaluate

metric = evaluate.load("wer")

We need to craft a function that processes our model predictions and calculates the WER metric. This function, named `compute_metrics`, initially substitutes `-100` with the `pad_token_id` in the `label_ids`, reversing the adjustment made in the data collator to accurately exclude padded tokens from the loss calculation. Subsequently, it translates the predicted and label ids into strings. Ultimately, it determines the WER by comparing the predictions with the reference labels:

In [ ]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

### Load a pre-trained checkpoint

Now, we'll proceed to load the pre-trained Whisper `small` checkpoint, which is straightforward with the use of 🤗 Transformers!

In [ ]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")
model.generation_config.language = "hi" # Define the language of your choice here

Adjust generation parameters - no tokens are predetermined as decoder outputs (refer to [`forced_decoder_ids`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.forced_decoder_ids)), and no tokens are excluded during the generation process (see [`suppress_tokens`](https://huggingface.co/docs/transformers/main_classes/text_generation#transformers.generation_utils.GenerationMixin.generate.suppress_tokens)):

In [ ]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

### Define the training configuration

For the concluding step, we specify every parameter associated with the training process. To dive deeper into the specifics of training arguments, consult the Seq2SeqTrainingArguments [documentation](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments).

In [ ]:
from huggingface_hub import whoami

whoami()
# you should see something like {'type': 'user',  'id': '...',  'name': 'Wauplin', ...}

In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./whisper-small-hi",  # change to a repo name of your choice
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=5e-5,
    # warmup_steps=500,
    # max_steps=4000,
    # warmup_steps=50,
    # max_steps=400,
    warmup_steps=10,
    max_steps=100,
    #num_train_epochs = 2.0,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=10,
    predict_with_generate=True,
    generation_max_length=225,
    # save_steps=1000,
    # eval_steps=1000,
    save_steps=100,
    eval_steps=100,
    logging_steps=10,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    hub_model_id = "jbatista79/20240410-small-hindi", # Change this to your own Hugging Face repo
    # hub_token="
    push_to_hub=True,
    # push_to_hub=False,
)

**Note**: if one does not want to upload the model checkpoints to the Hub,
set `push_to_hub=False`.

We can pass the training arguments to the 🤗 Trainer, together with our model, dataset, data collator, and `compute_metrics` function:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=common_voice["train"],
    eval_dataset=common_voice["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

Before initiating the training process, we'll save the processor object. Given that the processor is non-trainable, it will remain unchanged throughout the training period:

In [ ]:
processor.save_pretrained(training_args.output_dir)

### Training

The training duration is estimated to be around 5-10 hours, contingent on the capabilities of your GPU or the one provided by this Google Colab. If you're utilizing this Google Colab for fine-tuning a Whisper model, it's crucial to ensure that the training process does not get halted due to periods of inactivity. To circumvent this issue, you can employ a straightforward solution by inserting the following script into the console of this browser tab (by right-clicking -> selecting _inspect_ -> navigating to the _Console_ tab -> pasting the code).

```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton, 60000);
```

The maximum GPU memory usage for this training setup is roughly 15.8GB. If the GPU assigned to your Google Colab session is not sufficient, you may run into a CUDA `"out-of-memory"` error upon initiating the training. Should this occur, a viable strategy is to gradually decrease the `per_device_train_batch_size` by halves and utilize [`gradient_accumulation_steps`](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Seq2SeqTrainingArguments.gradient_accumulation_steps) as a balancing measure.

To begin the training process, just run:

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
locale.getpreferredencoding()


In [ ]:
trainer.train()

## Building a demo

Having fine-tuned our model, we're now ready to demonstrate its ASR prowess through a demo. We'll employ the 🤗 Transformers `pipeline` function, which seamlessly manages the entire ASR process, from preprocessing the audio inputs to decoding the model's predictions.

Executing the code below will create a Gradio demo. This demo allows us to capture speech using our computer's microphone and submit it to our fine-tuned Whisper model for text transcription:

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install gradio

In [ ]:
from transformers import pipeline
import gradio as gr

pipe = pipeline(model="jbatista79/20240410-small-hindi")  # change to "your-username/the-name-you-picked"

def transcribe(audio):
    text = pipe(audio)["text"]
    return text

iface = gr.Interface(
    fn=transcribe,
    # inputs=gr.Audio(source="microphone", type="filepath"),
    inputs=gr.Audio(sources=["microphone"], type="filepath"),
    outputs="text",
    title="Whisper Small Hindi",
    description="Realtime demo for Hindi speech recognition using a fine-tuned Whisper small model.",
)

iface.launch()

# Gratitude

Many thanks to Naval Katoch for his valuable technical insights. For more details on the Whisper model, the Common Voice dataset and the theory behind fine-tuning, check out the pioneering work by Hugging Face's Sanchit Gandhi: [blog post](https://huggingface.co/blog/fine-tune-whisper).